# Notebook for collecting and saving various subsets of the data

In [ ]:
import numpy as np
import os

### Collecting sniff and spiking data and saving it to a new directory

In [ ]:
# defining directories
spike_dir = r"E:\clickbait-ephys\kilosorted_data_20um_tetrode"
sniff_dir = r"C:\Users\smearlab\clickbait-ephys\preprocessed_data"
save_dir = r"D:\clickbait-ephys(3-5-25)"
os.makedirs(save_dir, exist_ok=True)

# defining parameters
mice = ['6000', '6001', '6002', '6003']


for mouse in mice:
    spike_sessions = os.listdir(os.path.join(spike_dir, mouse))
    sniff_sessions = os.listdir(os.path.join(sniff_dir, mouse))
    


    # check that the same number of sessions exist for each mouse
    assert len(spike_sessions) == len(sniff_sessions), f"Different number of sessions for {mouse}"

    # loop through sessions
    for spike_session, sniff_session in zip(spike_sessions, sniff_sessions):

        try:

            # creating a directory to save the data
            save_dir_session = os.path.join(save_dir, mouse, spike_session)
            os.makedirs(save_dir_session, exist_ok=True)

            # Getting the files to transfer
            sniff_file = os.path.join(sniff_dir, mouse, sniff_session, 'sniff.npy')
            spike_times_file = os.path.join(spike_dir, mouse, spike_session, 'spike_times.npy')
            spike_clusters_file = os.path.join(spike_dir, mouse, spike_session, 'spike_clusters.npy')
            spike_cluster_group_file = os.path.join(spike_dir, mouse, spike_session, 'cluster_group.tsv')

            # copy the data to the new directory
            os.system(f'copy {sniff_file} {save_dir_session}')
            os.system(f'copy {spike_times_file} {save_dir_session}')
            os.system(f'copy {spike_clusters_file} {save_dir_session}')


            # loading the templates to find the best channel for each unit
            template_path = os.path.join(spike_dir, mouse, spike_session, 'templates.npy')
            template_ind_path = os.path.join(spike_dir, mouse, spike_session, 'templates_ind.npy')
            templates = np.load(template_path)
            templates_ind = np.load(template_ind_path)

            # Find peak amplitude channel for each template and assign to unit
            peak_channels = np.argmax(np.max(np.abs(templates), axis=1), axis=1)
            unit_best_channels = {unit: templates_ind[unit, peak_channels[unit]] for unit in range(len(peak_channels))}

            # Separate OB and HC units
            hc_units = np.array([unit for unit, ch in unit_best_channels.items() if ch in range(0, 16)])
            ob_units = np.array([unit for unit, ch in unit_best_channels.items() if ch in range(16, 32)])
            
            # creating one text file to save the best channel for each unit and the region (OB vs HC)
            with open(os.path.join(save_dir_session, 'unit_best_channels.txt'), 'w') as f:
                f.write('unit,channel,region\n')
                for unit, ch in unit_best_channels.items():
                    region = 'HC' if unit in hc_units else 'OB'
                    f.write(f'{unit},{ch},{region}\n')

            
            print(f"Copied data for {mouse} {spike_session} to {save_dir_session}")
        except Exception as e:
            print(f"Error in {mouse} {spike_session}")
            print(e)
            continue

### Building sniff data directory

In [ ]:
sniff_params_dir = r"C:\Users\smearlab\Downloads\clickbait_ephys"
save_dir = r"E:\clickbait-ephys\data\sniff"

mice = ['6000', '6001', '6002', '6003']

for mouse in mice:
    mouse_dir = os.path.join(sniff_params_dir, mouse)
    sessions = os.listdir(mouse_dir)
    for session in sessions:
        session_dir = os.path.join(mouse_dir, session)
        files = os.listdir(session_dir)
        if 'sniff_params.mat' in files and 'sniff.npy' in files:
            save_dir_session = os.path.join(save_dir, mouse, session)
            os.makedirs(save_dir_session, exist_ok=True)
            os.system(f'copy {os.path.join(session_dir, "sniff_params.mat")} {save_dir_session}')
            os.system(f'copy {os.path.join(session_dir, "sniff.npy")} {save_dir_session}')
            print(f"Copied data for {mouse} {session} to {save_dir_session}")